In [1]:
from google.colab import drive
drive.mount("/content/drive")

# Global project paths
PROJECT = "/content/drive/MyDrive/rank-collapse"
# PROJECT = "/content/drive/MyDrive/rank-collapse"
%cd {PROJECT}

from utils import *

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/.shortcut-targets-by-id/1dwgmS-Fk3Kgg-RWfFuIvr8z2CnGXc00h/rank-collapse
🖥️  Using device: cuda
   GPU: NVIDIA A100-SXM4-40GB
✅ Setup complete!
✅ Datasets defined:
   - Copy (easy, diagonal attention)
   - Reverse (medium, anti-diagonal)
   - Sort (hard, global reasoning)
   - Text (language modeling)
   - Image (classification)
✅ Transformer architectures defined:
   - TINY: 2 layers, 64d, 4 heads (~0K params)
   - SMALL: 4 layers, 128d, 8 heads (~4K params)
   - MEDIUM: 6 layers, 256d, 8 heads (~12K params)
   - LARGE: 8 layers, 512d, 16 heads (~65K params)
✅ Rank metrics defined:
   - Stable rank (Frobenius / Operator norm)
   - Entropy effective rank
   - Top-k energy fraction
   - Attention sparsity
✅ Training system ready!
✅ Visualization tools ready!


In [ ]:
def make_standard_scheduler(which_layers="all"):
    return TrainingScheduler(
        which_layers=which_layers, base_func=cross_entropy,
        functions_at_times=[])

def run_noise(task_name, dataset_size_name, arch_name, when, weight=1.0, steps=2000, batch_size=64,
                      lr=3e-4, weight_decay=0.01, out_dir="logs", seed=42):
    """
    Build dataset, model, scheduler for the given config and run run_experiment.
    Saves logs as npy in out_dir with descriptive name.
    Returns path to saved file.
    """
    set_seed(seed)
    sizes = {"small": 1024, "medium": 4096, "large": 8192}
    dataset_size = sizes[dataset_size_name]

    # create datasets
    ds_train, task_type, vocab_size, _ = create_dataset(
        task_name, seq_len=32, vocab_size=64, dataset_size=dataset_size,
        split="train")
    ds_probe, _, _, _ = create_dataset(
        task_name, seq_len=32, vocab_size=64,
        dataset_size=min(512, dataset_size), split="train")

    # instantiate model
    vocab_size=max(32, vocab_size)
    seq_len=32
    arch_config = ARCHITECTURES[arch_name].copy()
    d_model = arch_config['d_model']
    nhead = arch_config['nhead']
    num_layers = arch_config['num_layers']
    add_noise = NoiseScheduler(1, 1, 2000)
    model = ConfigurableTransformer(vocab_size=vocab_size, d_model=d_model, nhead=nhead,
                                    num_layers=num_layers, dim_feedforward=arch_config['dim_feedforward'],
                                    seq_len=seq_len, use_pos=True, causal=(task_type!='classification'), add_noise=add_noise)
    model = model.to(device)

    # build scheduler
    scheduler = make_standard_scheduler()

    # run experiment; ensure run_experiment uses kwargs activations, layers, and model
    log_name = f"{task_name}_{dataset_size_name}_{arch_name}_noise_{when}_steps{steps}_seed{seed}.npy"
    os.makedirs(out_dir, exist_ok=True)
    out_path = os.path.join(out_dir, log_name)

    model, _ = run_experiment((ds_train, task_type, None, None),
                (ds_probe, task_type, None, None),
                model, arch_config,
                steps=steps, batch_size=batch_size,
                checkpoint_every=max(1, steps // 20), lr=lr,
                seed=seed, loss_scheduler=scheduler, metric_func=full_spectrum_metrics,
                log_store_name=out_path, weight_decay=weight_decay)

    print(f"Saved logs to {out_path}")
    return out_path

def run_normal(task_name, dataset_size_name, arch_name, when, weight=1.0, steps=2000, batch_size=64,
                      lr=3e-4, weight_decay=0.01, out_dir="logs", seed=42):
    """
    Build dataset, model, scheduler for the given config and run run_experiment.
    Saves logs as npy in out_dir with descriptive name.
    Returns path to saved file.
    """
    set_seed(seed)
    sizes = {"small": 1024, "medium": 4096, "large": 8192}
    dataset_size = sizes[dataset_size_name]

    # create datasets
    ds_train, task_type, vocab_size, _ = create_dataset(
        task_name, seq_len=32, vocab_size=64, dataset_size=dataset_size,
        split="train")
    ds_probe, _, _, _ = create_dataset(
        task_name, seq_len=32, vocab_size=64,
        dataset_size=min(512, dataset_size), split="train")

    # instantiate model
    vocab_size=max(32, vocab_size)
    seq_len=32
    arch_config = ARCHITECTURES[arch_name].copy()
    d_model = arch_config['d_model']
    nhead = arch_config['nhead']
    num_layers = arch_config['num_layers']
    model = ConfigurableTransformer(vocab_size=vocab_size, d_model=d_model, nhead=nhead,
                                    num_layers=num_layers, dim_feedforward=arch_config['dim_feedforward'],
                                    seq_len=seq_len, use_pos=True, causal=(task_type!='classification'))
    model = model.to(device)

    # build scheduler
    scheduler = make_standard_scheduler()

    # run experiment; ensure run_experiment uses kwargs activations, layers, and model
    log_name = f"{task_name}_{dataset_size_name}_{arch_name}_normal_{when}_steps{steps}_seed{seed}.npy"
    os.makedirs(out_dir, exist_ok=True)
    out_path = os.path.join(out_dir, log_name)
    if not os.path.exists(out_path):
        model, _ = run_experiment((ds_train, task_type, None, None),
                    (ds_probe, task_type, None, None),
                    model, arch_config,
                    steps=steps, batch_size=batch_size,
                    checkpoint_every=max(1, steps // 20), lr=lr,
                    seed=seed, loss_scheduler=scheduler, metric_func=full_spectrum_metrics,
                    log_store_name=out_path, weight_decay=weight_decay)

        print(f"Saved logs to {out_path}")
    else:
        print(f"{out_path} has already been done")
    return out_path

tasks = ["copy", "sort", "tiny_stories"]
archs = ["small", "medium", "large"]
dataset_sizes = ["medium", "large"]
whens = ["all"]

for task in tasks:
    for size_name in dataset_sizes:
        for arch in archs:
            for when in whens:
                print(f"RUN: task={task} size={size_name} arch={arch} when={when}")
                run_noise(task, size_name, arch, when,
                        steps=2000, batch_size=64, out_dir="logs", seed=42)

In [2]:
def run_single_config(task_name, dataset_size_name, arch_name, reg_name,
                      reg_func, when, weight=1.0, steps=2000, batch_size=64,
                      lr=3e-4, weight_decay=0.01, out_dir="logs", seed=42):
    """
    Build dataset, model, scheduler for the given config and run run_experiment.
    Saves logs as npy in out_dir with descriptive name.
    Returns path to saved file.
    """
    set_seed(seed)
    sizes = {"small": 1024, "medium": 4096, "large": 8192}
    dataset_size = sizes[dataset_size_name]

    # create datasets
    ds_train, task_type, vocab_size, _ = create_dataset(
        task_name, seq_len=32, vocab_size=64, dataset_size=dataset_size,
        split="train")
    ds_probe, _, _, _ = create_dataset(
        task_name, seq_len=32, vocab_size=64,
        dataset_size=min(512, dataset_size), split="train")

    # instantiate model
    model, arch_config = instantiate_architecture(
        arch_name, vocab_size=max(32, vocab_size), seq_len=32, task_type=task_type)
    model = model.to(device)

    # build scheduler
    scheduler = make_scheduler_for_regularizer(
        reg_func, steps=steps, when=when, which_layers="all", weight=weight)

    # run experiment; ensure run_experiment uses kwargs activations, layers, and model
    log_name = f"{task_name}_{dataset_size_name}_{arch_name}_{reg_name}_{when}_steps{steps}_seed{seed}.npy"
    os.makedirs(out_dir, exist_ok=True)
    out_path = os.path.join(out_dir, log_name)

    if not os.path.exists(log_name):
        model, _ = run_experiment((ds_train, task_type, None, None),
                    (ds_probe, task_type, None, None),
                    model, arch_config,
                    steps=steps, batch_size=batch_size,
                    checkpoint_every=max(1, steps // 20), lr=lr,
                    seed=seed, loss_scheduler=scheduler, metric_func=full_spectrum_metrics,
                    log_store_name=out_path, weight_decay=weight_decay)
        print(f"Saved logs to {out_path}")
    else:
        print(f"Already saved to {out_path}")
    return out_path


In [3]:
run_single_config("tiny_stories", "medium", "large", "logdet", increase_rank_regularizers["logdet"], "all")

Token indices sequence length is longer than the specified maximum sequence length for this model (1087 > 1024). Running this sequence through the model will result in indexing errors


Model: 8 layers, 512d, 16 heads
Parameters: 76,995,665
Device: cuda


Training: 100%|██████████| 2000/2000 [07:26<00:00,  4.48step/s, loss=4.9808, acc=0.2292]

✅ Training complete in 446.1s
Saved logs to logs/tiny_stories_medium_large_logdet_all_steps2000_seed42.npy


'logs/tiny_stories_medium_large_logdet_all_steps2000_seed42.npy'

In [4]:
run_single_config("tiny_stories", "medium", "large", "erank", increase_rank_regularizers["erank"], "all")

Model: 8 layers, 512d, 16 heads
Parameters: 76,995,665
Device: cuda


Training: 100%|██████████| 2000/2000 [14:42<00:00,  2.27step/s, loss=4.1182, acc=0.2550]

✅ Training complete in 882.8s
Saved logs to logs/tiny_stories_medium_large_erank_all_steps2000_seed42.npy


'logs/tiny_stories_medium_large_erank_all_steps2000_seed42.npy'

In [5]:
run_single_config("tiny_stories", "medium", "large", "orthonorm", increase_rank_regularizers["orthonorm"], "all")

Model: 8 layers, 512d, 16 heads
Parameters: 76,995,665
Device: cuda


Training: 100%|██████████| 2000/2000 [06:55<00:00,  4.81step/s, loss=2.7402, acc=0.4253]

✅ Training complete in 415.9s
Saved logs to logs/tiny_stories_medium_large_orthonorm_all_steps2000_seed42.npy


'logs/tiny_stories_medium_large_orthonorm_all_steps2000_seed42.npy'

In [8]:
run_single_config("tiny_stories", "medium", "large", "spectral_norm", decrease_rank_regularizers["spectral_norm"], "all")

Model: 8 layers, 512d, 16 heads
Parameters: 76,995,665
Device: cuda


Training: 100%|██████████| 2000/2000 [22:27<00:00,  1.48step/s, loss=2.7461, acc=0.4255]

✅ Training complete in 1347.6s
Saved logs to logs/tiny_stories_medium_large_spectral_norm_all_steps2000_seed42.npy


'logs/tiny_stories_medium_large_spectral_norm_all_steps2000_seed42.npy'